In [ ]:
import numpy as np

f_dev_embedding_path_pattern = "/data/user_data/sbharad2/SpeechCLIP/data/Fastvgs_embeddings/embeddings_flickr_dev_matrix_snr{snr}.npy"

for snr in range(5, 41, 5):
    print(f"SNR: {snr}")
    f_dev_embedding_path = f_dev_embedding_path_pattern.format(snr=snr)
    f_dev_txt_path = "/data/user_data/sbharad2/SpeechCLIP/data/Fastvgs_embeddings/embeddings_flickr_dev_clean.txt"

    f_dev_img_embedding_path = (
        "/data/user_data/sbharad2/SpeechCLIP/data/Fastvgs_embeddings/img_embeddings.npy"
    )
    f_dev_img_txt_path = (
        "/data/user_data/sbharad2/SpeechCLIP/data/Fastvgs_embeddings/img_ids.txt"
    )

    f_dev_speech_emb = np.load(f_dev_embedding_path)
    f_dev_img_emb = np.load(f_dev_img_embedding_path)

    def _transform_wav_name_to_example_id(wav_name):
        wav_name = wav_name.replace(".wav", "")
        caption_id = wav_name[-1]
        img_id = wav_name[:-2]
        return f"{img_id}.jpg#{caption_id}"

    with open(f_dev_txt_path, "r") as f:
        f_dev_speech_names = [line.strip() for line in f.readlines()]
        f_dev_speech_names = [
            _transform_wav_name_to_example_id(name) for name in f_dev_speech_names
        ]

    f_dev_speech_emb_dict = {
        name: emb for name, emb in zip(f_dev_speech_names, f_dev_speech_emb)
    }
    # normalize
    f_dev_speech_emb_dict = {
        k: v / np.linalg.norm(v) for k, v in f_dev_speech_emb_dict.items()
    }

    with open(f_dev_img_txt_path, "r") as f:
        f_dev_img_names = [line.strip() for line in f.readlines()]

    # Load image embeddings
    f_dev_img_emb_dict = {
        name: emb for name, emb in zip(f_dev_img_names, f_dev_img_emb)
    }
    # normalize
    f_dev_img_emb_dict = {
        k: v / np.linalg.norm(v) for k, v in f_dev_img_emb_dict.items()
    }

    print(len(f_dev_speech_emb_dict), len(f_dev_img_emb_dict))

    # Load dev df
    import pandas as pd

    df = pd.read_csv(
        "/data/user_data/sbharad2/SpeechCLIP/data/flickr/flickr_dev_sampled.csv"
    )

    def retrieve_image_scores(speech_embedding, image_embeddings, topk=10):
        scores = {k: np.dot(speech_embedding, v) for k, v in image_embeddings.items()}
        retrieved_imgs = [
            k
            for k, v in sorted(scores.items(), key=lambda item: item[1], reverse=True)[
                :topk
            ]
        ]
        return retrieved_imgs

    # Compute Recall@k
    predictions = {}
    for i, r in df.iterrows():
        example_id = r["example_id"]
        predictions[example_id] = retrieve_image_scores(
            f_dev_speech_emb_dict[example_id], f_dev_img_emb_dict
        )

    recall = {1: 0, 5: 0, 10: 0}
    for k, v in predictions.items():
        for topk in [1, 5, 10]:
            if k.split("#")[0] in v[:topk]:
                recall[topk] += 1

    recall = {k: v / len(predictions) for k, v in recall.items()}
    print(recall)

SNR: 5
1000 200
{1: 0.17, 5: 0.458, 10: 0.584}
SNR: 10
1000 200
{1: 0.3, 5: 0.644, 10: 0.763}
SNR: 15
1000 200
{1: 0.361, 5: 0.712, 10: 0.821}
SNR: 20
1000 200
{1: 0.397, 5: 0.754, 10: 0.855}
SNR: 25
1000 200
{1: 0.418, 5: 0.77, 10: 0.865}
SNR: 30
1000 200
{1: 0.422, 5: 0.768, 10: 0.871}
SNR: 35
1000 200
{1: 0.428, 5: 0.771, 10: 0.876}
SNR: 40
1000 200
{1: 0.427, 5: 0.783, 10: 0.885}
